In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# the service URL
livechart = "https://nds.iaea.org/relnsd/v0/data?"


In [2]:
import urllib.request

def lc_read_csv(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

In [3]:
# load data into a dataframe 

# this loading might not work, then use the line below
#df = pd.read_csv(livechart + "fields=decay_rads&nuclides=241am&rad_types=g")
df = lc_read_csv(livechart + "fields=decay_rads&nuclides=241am&rad_types=g")

df = df[pd.to_numeric(df['intensity'],errors='coerce').notna()] # remove blanks (unknown intensities)
df.intensity = df['intensity'].astype(float) # convert to numeric. Note how one can specify the field by attribute or by string 

fig = px.scatter(df, x="energy", y="intensity", log_y=True) # plot in log scale
fig.show()

In [4]:
url = livechart+"fields=decay_rads&nuclides=100nb&rad_types=bm"

# this loading might not work, then use the line below
#df=pd.read_csv(url)[['transition_type','log_ft']] # optional:load only transition_type and log_ft 
df=lc_read_csv(url)[['transition_type','log_ft']] # optional:load only transition_type and log_ft 

df = df[pd.to_numeric(df.log_ft,errors='coerce').notna()] # remove unknowns and convert to numeric values
df.log_ft = df.log_ft.astype(float)

ans = [pd.DataFrame(y) for x, y in df.groupby('transition_type', as_index=False)] # group log ft values by transition
fig = go.Figure()

for a in ans:
    ct = pd.cut(a.log_ft, bins=10)   # bin each grouping 
    tst = ct.apply(lambda x: x.mid).value_counts(sort=False)  # assign the mid value to each bin
    fig.add_trace(go.Scatter(x=tst.index.values, y = tst, mode="markers", line_shape='spline', name=a.transition_type.iloc[0]))
fig.show()

In [5]:
url=livechart+"fields=ground_states&nuclides=all"
# this loading might not work, then use the line below
#df=pd.read_csv(url)[['symbol','n','half_life_sec']].query('symbol=="Ac"') # further filter
df=lc_read_csv(url)[['symbol','n','half_life_sec']].query('symbol=="Ac"') # further filter

fig = px.scatter(df, x="n", y="half_life_sec", log_y=True)
fig.show()

In [8]:
url=livechart+"fields=cumulative_fy&parents=235u"
# this loading might not work, then use the line below
#df = pd.read_csv(url) [['z_daughter','a_daughter',"element_daughter",'cumulative_fast_fy']]
df = lc_read_csv(url) [['z_daughter','a_daughter',"element_daughter",'cumulative_fast_fy']]
df = df[pd.to_numeric(df.cumulative_fast_fy,errors='coerce').notna()]
df.cumulative_fast_fy = df.cumulative_fast_fy.astype(float)
  

fig = go.Figure(data=[go.Scatter3d(
    x=df["z_daughter"],
    y=df["a_daughter"]-df["z_daughter"],
    z=df["cumulative_fast_fy"],
    mode='markers',
    text = df['a_daughter'].apply(str) + df["element_daughter"],
    marker=dict(
        size=7
        ,color=df["cumulative_fast_fy"] # array/list of values setting the color
        ,colorscale='Viridis'           # colorscale
    )
     

)])

camera = dict(                          # initial spacial settings 
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0.1, y=2, z=-1.5)        # view it from the bottom
)

fig.update_layout(scene_camera=camera, margin=dict(l=0, r=0, b=0, t=0), height=900)
fig.show()

# 2D plot
fig2 = go.Figure()
fig2.add_trace(go.Scatter( x=df[df.columns[1]], y=df['cumulative_fast_fy'], mode="markers", text= df['a_daughter'].apply(str) + df["element_daughter"]))
fig2.update_layout( height=1000, xaxis_title="A", yaxis_title="Yield ") 
fig2.show()










In [10]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", 3)

# GS
df=pd.read_csv(livechart+"fields=ground_states&nuclides=236np")
df

HTTPError: ignored